In [1]:
from mIAm.graph.workflow import workflow
from mIAm.graph.trimmer import TRIMMER
import os 
from psycopg_pool import AsyncConnectionPool
from langgraph.checkpoint.postgres.aio import AsyncPostgresSaver
from langchain_core.messages import HumanMessage


In [2]:
DB_URI = f"postgresql://{os.getenv('PSQL_USERNAME')}:{os.getenv('PSQL_PASSWORD')}" \
         f"@{os.getenv('PSQL_HOST')}:{os.getenv('PSQL_PORT')}/{os.getenv('PSQL_DATABASE')}" \
         f"?sslmode={os.getenv('PSQL_SSLMODE')}"
            
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}

user_config = {
    "first_name": "Branis",
    "last_name": "GHOUL",
    "email": "branisghoul02@hotmail.com",
    "password": "123456789",
    "phone": "123456789",
    "address": "30 rue victor hugo",
    "city": "Paris",
    "country": "France",
    "zip_code": "75001",
    "language": ["fr", "en", "ar"],
    "timezone": "Europe/Paris",
    "currency": "EUR",
}



In [3]:
async with AsyncConnectionPool(
    conninfo=DB_URI,
    max_size=20,
    kwargs=connection_kwargs
) as pool:
    checkpointer = AsyncPostgresSaver(pool)

    # NOTE: you need to call .setup() the first time you're using your checkpointer
    await checkpointer.setup()
    graph = workflow.compile(checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "1"}}
    while True:
        user = input("User (q/Q to quit): ")
        print(f"User (q/Q to quit): {user}")
        if user in {"q", "Q"}:
            print("AI: Byebye")
            break
        output = None
        async for output in graph.astream(
            {"messages": [HumanMessage(content=user)]}, config=config, stream_mode="updates"
        ):
            last_message = next(iter(output.values()))["messages"][-1]
            last_message.pretty_print()

        if output and "prompt" in output:
            print("Done!")
            

           
    checkpoint = await checkpointer.aget(config)

User (q/Q to quit): hi
================================== Ai Message ==================================

Hi there! How can I assist you today with your cooking or recipe needs? Whether you're looking for a new dish to try, need some cooking tips, or have a specific question, feel free to ask! 🍳🥗
User (q/Q to quit): tajin zitoun
================================== Ai Message ==================================

Tajine Zitoun is a delicious Algerian dish featuring chicken, olives, and a flavorful sauce. It's a comforting and aromatic meal that's perfect for any occasion. Here's a simple recipe to guide you:

### Tajine Zitoun Recipe

#### Ingredients:
- 1 kg chicken pieces (thighs or drumsticks work well)
- 2 cups green olives, pitted
- 1 large onion, finely chopped
- 3 cloves garlic, minced
- 2 tablespoons olive oil
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon paprika
- 1/2 teaspoon turmeric
- Salt and pepper to taste
- 1 lemon, juiced
- 2 cups chicken broth or wat

In [4]:
trimmed_message = TRIMMER.invoke(checkpoint["channel_values"]["messages"])
len(trimmed_message), trimmed_message

(13,
 [HumanMessage(content='', additional_kwargs={}, response_metadata={}, id='6b1bdd8c-3a07-4c76-8ebe-fb42eac48881'),
  AIMessage(content="It seems like your message was empty again. If you have a question or need assistance with anything related to cooking or recipes, feel free to type it out. I'm here to help! 🍽️👩\u200d🍳", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 434, 'total_tokens': 481, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'stop', 'logprobs': None}, id='run-46ec59cf-67cf-4210-86c0-d3848eaac810-0', usage_metadata={'input_tokens': 434, 'output_tokens': 47, 'total_tokens': 481, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {